### Part 1.12 - Extract_data_58098_movies

#### Import the libraries

In [ ]:
# For cleaning and preparing the dataset
# -> dataframe manipulation
# -> text manipulation
# -> Web Scrapping

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import os

# Module to serialize the content produced from the execution of the code

import pickle

# Module to monitor the progress of a python for loop

from tqdm import tqdm_notebook
# Example of Use: tqdm_notebook(examples, desc="Converting examples to features")

#### Last 3098 movies (58,098 in total)

Extract data about:

* field 1: Plot Summary
* field 2: Actors
* field 3: Directors
* field 4: IMDB rating
* field 5: Plot Synopsis
* field 6: Reviews

In [ ]:
"""
Open the movie_content urls for the twelfth 5000 movies and create the souplist of those URLs
"""
content_twelve = []

with open(os.path.join(os.getcwd(), 'movie_content_url\\data_twelve_14012020.pkl'), 'rb') as f:
    
    content_twelve = pickle.load(f)

print("Number of URLs: {}".format(len(content_twelve)))

content_souplist_twelve = []

for i in tqdm_notebook(content_twelve):
    
    content_souplist_twelve.append(BeautifulSoup(i.text))

print("Number of souplists: {}".format(len(content_souplist_twelve)))

In [ ]:
"""
Open the synopsis urls of the twelfth 5000 movies and create the souplist of those URLs
"""
synopsis_twelve = []

with open(os.path.join(os.getcwd(), 'synopsis_url\\synopsis_twelve_14012020.pkl'), 'rb') as f:
    
    synopsis_twelve = pickle.load(f)

print("Number of URLs: {}".format(len(synopsis_twelve)))

synopsis_souplist_twelve = []

for i in tqdm_notebook(synopsis_twelve):
    
    synopsis_souplist_twelve.append(BeautifulSoup(i.text))

print("Number of souplists: {}".format(len(synopsis_souplist_twelve)))

In [ ]:
"""
Open the reviews urls of the twelfth 5000 movies and create the souplist of those URLs
"""
review_twelve = []

with open(os.path.join(os.getcwd(), 'reviews_url\\review_twelve_16012020.pkl'), 'rb') as f:
    
    review_twelve = pickle.load(f)

print("Number of URLs: {}".format(len(review_twelve)))

review_souplist_twelve = []

for i in tqdm_notebook(review_twelve):
    
    review_souplist_twelve.append(BeautifulSoup(i.text))

print("Number of review tags: {}".format(len(review_souplist_twelve)))

In [ ]:
"""
Field 1: Extract plot summary
"""
myfield_plot = []
plot_summary = []
index_to_remove_no_plot = []

[myfield_plot.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_twelve)]

[[[plot_summary.append(y.text) for y in x.find_all('div', {'class':'summary_text'})] for x in i] if len(i) !=0 else index_to_remove_no_plot.append(myfield_plot.index(i)) for i in myfield_plot]
      
print("Length of Plot Summary list: {}".format(len(plot_summary)))
print("Length of the list with Movies that don't have plot summary: {}".format(len(index_to_remove_no_plot)))
if len(index_to_remove_no_plot) == 0:
    print("None of the movie miss plot")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_plot))

In [ ]:
"""
Field 2: Extract actors
"""
myfield_cast = []
phase_two = []
phase_three = []
actors_list = []
index_to_remove_no_actors = []

[myfield_cast.append(i.find_all('table', {'class':'cast_list'})) for i in tqdm_notebook(content_souplist_twelve)]
    
r_one = re.compile(".*name")

[[phase_two.append(j.find_all('a', {'href':r_one})) for j in i] for i in myfield_cast]

[phase_three.append(phase_two[i][1::2]) for i in range(len(phase_two))]

[actors_list.append(list(map(lambda x: x.text.strip(' ').replace('\n', ''), actors))) for actors in phase_three]            
    
index_to_remove_no_actors = [i for i,x in enumerate(myfield_cast) if not x]
        
print("Length of Actors list: {}".format(len(actors_list)))
print("Length of the list with Movies that don't have actors: {}".format(len(index_to_remove_no_actors)))
if len(index_to_remove_no_actors) == 0:
    print("None of the movie miss actors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_actors))

In [ ]:
"""
Field 3: Extract director name(s)
"""
myfield_director = []
director_name = []
index_to_remove_no_directors = []

[myfield_director.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_twelve)]

r_name = re.compile(".*name")

[[director_name.append(j.find_all('a', {'href':r_name})) for j in i] for i in myfield_director]
    
director_names = [item[0].text for item in director_name if len(item)!=0]

index_to_remove_no_directors = [i for i,x in enumerate(myfield_director) if not x]

print("Length of Directors list: {}".format(len(director_names)))
print("Length of the list with Movies that don't have directors: {}".format(len(index_to_remove_no_directors)))
if len(index_to_remove_no_directors) == 0:
    print("None of the movie miss directors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_directors))

In [ ]:
"""
Field 4: Extract imdb movie rating
"""
myfield_rating = []
ratings = []
index_to_remove_no_rating = []

[myfield_rating.append(i.find_all('div', {'class':'ratingValue'})) for i in tqdm_notebook(content_souplist_twelve)]

[[[ratings.append(y.text) for y in x.find_all('span', {'itemprop':'ratingValue'})] for x in i] for i in myfield_rating]

index_to_remove_no_rating = [i for i,x in enumerate(myfield_rating) if not x]

print("Length of Ratings list: {}".format(len(ratings)))
print("Length of the list with Movies that are not rated: {}".format(len(index_to_remove_no_rating)))
if len(index_to_remove_no_rating) == 0:
    print("None of the movie miss ratings")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_rating))

In [ ]:
"""
Field 5: Extract plot synopsis

Having extracted the synopsis content for each movie, we noticed that roughly more than 50% percent of the movies have a synopsis text.
Thus, in order not to erase almost half of the first 5,000 we will not further proceed in keeping the synopsis text
"""
synopsis_step_one = []
synopsis_step_two = []
synopsis_step_three = []

[synopsis_step_one.append(i.find_all('ul', {'class':'ipl-zebra-list', 'id':'plot-synopsis-content'})) for i in tqdm_notebook(synopsis_souplist_twelve)]

[[synopsis_step_two.append(j.find_all('li', {'class':'ipl-zebra-list__item'})) for j in i] for i in synopsis_step_one]

[[synopsis_step_three.append(j.text.strip(' ').replace('\n', '').replace('\\', '')) for j in i] for i in synopsis_step_two]

index_to_remove_no_synopsis = [i for i,x in enumerate(synopsis_step_one) if not x]

print("Length of Synopsis list: {}".format(len(synopsis_step_three)))
print("Length of the list with Movies that don't have synopsis: {}".format(len(index_to_remove_no_synopsis)))
if len(index_to_remove_no_synopsis) == 0:
    print("None of the movies miss a synopsis")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_synopsis))

In [ ]:
"""
Print the movies with no synopsis text.
Result: We observed that many movies missed a synopsis text. Thus, we decided to not use synopsis for furhter analysis (although will be extracted)
"""
print([i for i,x in enumerate(synopsis_step_one) if not x])
print([i for i,x in enumerate(synopsis_step_two) if not x])
print([i for i,x in enumerate(synopsis_step_three) if not x])

# Many of the movies do not have a synopsis text, this is it won't be used for furhter analysis (althpugh will be extracted)

In [ ]:
"""
Field 6: Extract movie reviews
"""
myfield_review_step_one = []
myfield_review_step_two = []
myfield_review_step_three = []

[myfield_review_step_one.append(i.find_all('div', {'class':'lister-list'})) for i in tqdm_notebook(review_souplist_twelve)]

[[myfield_review_step_two.append(j.find_all('div', {'class':'text show-more__control'})) for j in i] for i in myfield_review_step_one]

[myfield_review_step_three.append(list(map(lambda x: x.text, reviews))) for reviews in myfield_review_step_two]

index_to_remove_no_review = [i for i,x in enumerate(myfield_review_step_one) if not x]

print("Length of Reviews list: {}".format(len(myfield_review_step_three)))
print("Length of the list with Movies that don't have reviews: {}".format(len(index_to_remove_no_review)))
if len(index_to_remove_no_review) == 0:
    print("None of the movies miss reviews")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_review))

In [ ]:
"""
Print the movies with no user reviews text.
"""
print([i for i,x in enumerate(myfield_review_step_one) if not x])
print([i for i,x in enumerate(myfield_review_step_two) if not x])
print([i for i,x in enumerate(myfield_review_step_three) if not x])

<b> - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - </b>

#### Dataframe creation based on the movie content, synopsis and reviews extracted

In [ ]:
"""
Import the movie_content (the extracted HTML document of the column imdb_url)
Import the reviews content (the extracted HTML document of the column reviews_url)
"""
with open(os.path.join(os.getcwd(), 'movie_content_url\\data_twelve_14012020.pkl'), 'rb') as f:
    
    content_twelve = pickle.load(f)

print("Number of URLs: {}".format(len(content_twelve)))

with open(os.path.join(os.getcwd(), 'reviews_url\\review_twelve_16012020.pkl'), 'rb') as f:
    
    review_twelve = pickle.load(f)

print("Number of URLs: {}".format(len(review_twelve)))

In [ ]:
"""
Below we import the 5 fields extracted previously
However, we we imported the "rating" file, we noticed that they did not downloaded properly. Actually, the respective list was empty,
thus we decided to download again the data in order no to make any other mistake with the index of the movies.
"""
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_twelve\\pre-indexed files\\plot_twelve_18012020.pkl'), 'rb') as f:
    
    plot = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_twelve\\pre-indexed files\\rating_twelve_18012020.pkl'), 'rb') as f:
    
    rating = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_twelve\\pre-indexed files\\actors_twelve_18012020.pkl'), 'rb') as f:
    
    actors = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_twelve\\pre-indexed files\\director_twelve_18012020.pkl'), 'rb') as f:
    
    director = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_twelve\\pre-indexed files\\synopsis_twelve_18012020.pkl'), 'rb') as f:
    
    synopsis = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_twelve\\pre-indexed files\\reviews_twelve_18012020.pkl'), 'rb') as f:
    
    reviews = pickle.load(f)
    
print(len(plot))
print(len(rating))
print(len(actors))
print(len(director))
print(len(synopsis))
print(len(reviews))

print(rating) #empty

In [ ]:
"""
Running the commands below we can spot the number of indexes that should be removed from the data extracted.
"Add a Plot": Denoted that the movies does have a written plot on its IMDB website.
"It looks like we don\'t have a Synopsis for this title yet. Be the first to contribute! Just click the "Edit page" button at the bottom of the page or learn more in the Synopsis submission guide.": Also denotes a non-written synopsis.

Synopsis indexes are not further considered. There are too many movies with no synopsis text and there is no poin in erasing almost half of the first 5000 movies due to missing synopsis text.
"""
matching_add_plot = [s for s, x in enumerate(plot) if "Add a Plot" in x]

print("Number of movies with 'Add Plot': ", len(matching_add_plot), '\n')

matching_add_synopsis = [s for s, x in enumerate(synopsis) if 'It looks like we don\'t have a Synopsis for this title yet. Be the first to contribute! Just click the "Edit page" button at the bottom of the page or learn more in the Synopsis submission guide.' in x]

print("Number of movies with no synopsis text: ", len(matching_add_synopsis))

In [ ]:
"""
Remove the faulty indexes from the data. As faulty is characterized an index with no data relevant to movie content or user review of a movie or with an error 404 web page.
Indexes collected during the data extraction of the 6 fields and from the above python cell that prints number of movies with no written plot.
"""
index_remove=[185,189,196,200,201,235,292,293,370,398,402,425,431,436,437,440,445,451,457,461,462,466,469,473,476,477,488,493,535,664,695,763,783,829,840,849,858,868,883,938,939,951,995,1003,1034,1040,1043,1070,1088,1120,1150,1151,1152,1163,1170,1171,1176,1178,1259,1274,1352,1359,1367,1368,1397,1406,1472,1474,1478,1479,1480,1482,1483,1487,1526,1560,1570,1586,1597,1606,1640,1646,1648,1660,1684,1701,1717,1760,1779,1817,1853,1858,1917,1923,1938,1988,2114,2118,2132,2138,2144,2150,2158,2194,2277,2303,2341,2342,2347,2349,2352,2366,2467,2500,2550,2607,2644,2656,2728,2729,2768,2793,2801,2825,2858,2894,2903,2940,2950,2992,2995,3014,3018,265,999,1064,1120,1170,1347,1478,1693,1788,1988,2051,2342,2347,2349,2350,2352,2353,2861,3024,3062,189,476,535,664,1170,1472,1760,1988,2656,19882,18,52,54,110,172,176,179,208,230,254,272,283,287,363,402,451,476,513,567,571,577,599,605,613,614,617,618,619,644,647,683,753,776,777,778,780,788,789,793,806,808,809,817,822,834,912,935,936,937,974,981,1030,1050,1083,1120,1151,1152,1161,1170,1172,1200,1266,1268,1270,1271,1274,1275,1276,1277,1308,1309,1341,1351,1368,1375,1380,1402,1412,1446,1450,1454,1458,1476,1518,1526,1527,1566,1569,1626,1641,1693,1702,1724,1729,1754,1769,1788,1800,1804,1813,1816,1823,1916,1918,1938,1959,1960,1965,1998,2029,2050,2120,2137,2152,2174,2286,2297,2312,2341,2347,2351,2352,2378,2421,2454,2460,2473,2507,2527,2533,2556,2571,2655,2660,2669,2683,2760,2858,2860,2898,2899,2934,2939,2989,3018,3049,3050,3051]
index_remove = list(dict.fromkeys(index_remove))
print(len(index_remove))
content_index_twelve = [i for j, i in enumerate(content_twelve) if j not in index_remove]
review_index_twelve = [i for j, i in enumerate(review_twelve) if j not in index_remove]
print(len(content_index_twelve))
print(len(review_index_twelve))

In [ ]:
"""
Run this cell once! For every re-execution of this notebook, please use the already pickled file.
"""
# with open(os.path.join(os.getcwd(), '58,000 movies\\movie_twelve\\content_index_twelve_20012020.pkl'), 'wb') as f:
#     pickle.dump(content_index_twelve, f)
    
# with open(os.path.join(os.getcwd(), '58,000 movies\\movie_twelve\\review_index_twelve_20012020.pkl'), 'wb') as f:
#     pickle.dump(review_index_twelve, f)

In [ ]:
"""
Import the pruned list of movie_content & reviews web pages. The indexes that have been removed did not contain one or more -
of the following fields: plot, actors, director(s), imdb_rating, review(s)

It is important to re-run the process of extracting the information in order to assure that we have extracted the correct information and text for each movie
Otherwise, we might observe mismatches in the fields assigned to each movie title.
"""
content_souplist_twelve = []

with open(os.path.join(os.getcwd(), '58,000 movies\\movies_twelve\\content_index_twelve_20012020.pkl'), 'rb') as f:
    
    content_index_twelve = pickle.load(f)

print("Number of URLs: {}".format(len(content_index_twelve)))

for i in tqdm_notebook(content_index_twelve):
    
    content_souplist_twelve.append(BeautifulSoup(i.text))

print("Number of souplists: {}".format(len(content_souplist_twelve)))

#-----------------------------------------------------------------------------------------------------

with open(os.path.join(os.getcwd(), '58,000 movies\\movies_twelve\\review_index_twelve_20012020.pkl'), 'rb') as f:
    
    review_twelve = pickle.load(f)

print("Number of URLs: {}".format(len(review_twelve)))

review_souplist_twelve = []

for i in tqdm_notebook(review_twelve):
    
    review_souplist_twelve.append(BeautifulSoup(i.text))

print("Number of review tags: {}".format(len(review_souplist_twelve)))

In [ ]:
"""
Field 1: Extract plot summary
"""
myfield_plot = []
plot_summary = []
index_to_remove_no_plot = []

[myfield_plot.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_twelve)]

[[[plot_summary.append(y.text) for y in x.find_all('div', {'class':'summary_text'})] for x in i] if len(i) !=0 else index_to_remove_no_plot.append(myfield_plot.index(i)) for i in myfield_plot]
      
print("Length of Plot Summary list: {}".format(len(plot_summary)))
print("Length of the list with Movies that don't have plot summary: {}".format(len(index_to_remove_no_plot)))
if len(index_to_remove_no_plot) == 0:
    print("None of the movie miss plot")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_plot))

In [ ]:
"""
Field 2: Extract actors
"""
myfield_cast = []
phase_two = []
phase_three = []
actors_list = []
index_to_remove_no_actors = []

[myfield_cast.append(i.find_all('table', {'class':'cast_list'})) for i in tqdm_notebook(content_souplist_twelve)]
    
r_one = re.compile(".*name")

[[phase_two.append(j.find_all('a', {'href':r_one})) for j in i] for i in myfield_cast]

[phase_three.append(phase_two[i][1::2]) for i in range(len(phase_two))]

[actors_list.append(list(map(lambda x: x.text.strip(' ').replace('\n', ''), actors))) for actors in phase_three]            
    
index_to_remove_no_actors = [i for i,x in enumerate(myfield_cast) if not x]
        
print("Length of Actors list: {}".format(len(actors_list)))
print("Length of the list with Movies that don't have actors: {}".format(len(index_to_remove_no_actors)))
if len(index_to_remove_no_actors) == 0:
    print("None of the movie miss actors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_actors))

In [ ]:
"""
Field 3: Extract director name(s)
"""
myfield_director = []
director_name = []
index_to_remove_no_directors = []

[myfield_director.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_twelve)]

r_name = re.compile(".*name")

[[director_name.append(j.find_all('a', {'href':r_name})) for j in i] for i in myfield_director]
    
director_names = [item[0].text for item in director_name]

index_to_remove_no_directors = [i for i,x in enumerate(myfield_director) if not x]

print("Length of Directors list: {}".format(len(director_names)))
print("Length of the list with Movies that don't have directors: {}".format(len(index_to_remove_no_directors)))
if len(index_to_remove_no_directors) == 0:
    print("None of the movie miss directors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_directors))

In [ ]:
"""
Field 4: Extract imdb movie rating
"""
myfield_rating = []
ratings = []
index_to_remove_no_rating = []

[myfield_rating.append(i.find_all('div', {'class':'ratingValue'})) for i in tqdm_notebook(content_souplist_twelve)]

[[[ratings.append(y.text) for y in x.find_all('span', {'itemprop':'ratingValue'})] for x in i] for i in myfield_rating]

index_to_remove_no_rating = [i for i,x in enumerate(myfield_rating) if not x]

print("Length of Ratings list: {}".format(len(ratings)))
print("Length of the list with Movies that are not rated: {}".format(len(index_to_remove_no_rating)))
if len(index_to_remove_no_rating) == 0:
    print("None of the movie miss ratings")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_rating))

In [ ]:
"""
Field 5: Extract movie user reviews
"""
myfield_review_step_one = []
myfield_review_step_two = []
myfield_review_step_three = []

[myfield_review_step_one.append(i.find_all('div', {'class':'lister-list'})) for i in tqdm_notebook(review_souplist_twelve)]

[[myfield_review_step_two.append(j.find_all('div', {'class':'text show-more__control'})) for j in i] for i in myfield_review_step_one]

[myfield_review_step_three.append(list(map(lambda x: x.text, reviews))) for reviews in myfield_review_step_two]

index_to_remove_no_review = [i for i,x in enumerate(myfield_review_step_one) if not x]

print("Length of Reviews list: {}".format(len(myfield_review_step_three)))
print("Length of the list with Movies that don't have reviews: {}".format(len(index_to_remove_no_review)))
if len(index_to_remove_no_review) == 0:
    print("None of the movies miss reviews")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_review))

In [ ]:
print(len([i for i,x in enumerate(myfield_review_step_one) if not x]))

print([i for i,x in enumerate(myfield_review_step_two) if not x])

print([i for i,x in enumerate(myfield_review_step_three) if not x])

In [ ]:
"""
Run this cell once! For every re-execution of this notebook, please use the already pickled file.
"""
# # Pickle the requests file for further use!

# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_twelve\\plot_twelve_20012020.pkl'), 'wb') as f:
#     pickle.dump(plot_summary, f)
    
# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_twelve\\actors_twelve_20012020.pkl'), 'wb') as f:
#     pickle.dump(actors_list, f)
    
# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_twelve\\director_twelve_20012020.pkl'), 'wb') as f:
#     pickle.dump(director_names, f)

# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_twelve\\reviews_twelve_24012020.pkl'), 'wb') as f:
#     pickle.dump(myfield_review_step_twelve, f)

In [ ]:
# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_twelve\\rating_twelve_20012020.pkl'), 'wb') as f:
#     pickle.dump(ratings, f)

In [ ]:
"""
Remove the faulty indexes from the data. As faulty is characterized an index with no data relevant to movie content or user review of a movie or with an error 404 web page.
Indexes collected during the data extraction of the 6 fields and from the above python cell that prints number of movies with no written plot.
"""
index_remove=[185,189,196,200,201,235,292,293,370,398,402,425,431,436,437,440,445,451,457,461,462,466,469,473,476,477,488,493,535,664,695,763,783,829,840,849,858,868,883,938,939,951,995,1003,1034,1040,1043,1070,1088,1120,1150,1151,1152,1163,1170,1171,1176,1178,1259,1274,1352,1359,1367,1368,1397,1406,1472,1474,1478,1479,1480,1482,1483,1487,1526,1560,1570,1586,1597,1606,1640,1646,1648,1660,1684,1701,1717,1760,1779,1817,1853,1858,1917,1923,1938,1988,2114,2118,2132,2138,2144,2150,2158,2194,2277,2303,2341,2342,2347,2349,2352,2366,2467,2500,2550,2607,2644,2656,2728,2729,2768,2793,2801,2825,2858,2894,2903,2940,2950,2992,2995,3014,3018,265,999,1064,1120,1170,1347,1478,1693,1788,1988,2051,2342,2347,2349,2350,2352,2353,2861,3024,3062,189,476,535,664,1170,1472,1760,1988,2656,19882,18,52,54,110,172,176,179,208,230,254,272,283,287,363,402,451,476,513,567,571,577,599,605,613,614,617,618,619,644,647,683,753,776,777,778,780,788,789,793,806,808,809,817,822,834,912,935,936,937,974,981,1030,1050,1083,1120,1151,1152,1161,1170,1172,1200,1266,1268,1270,1271,1274,1275,1276,1277,1308,1309,1341,1351,1368,1375,1380,1402,1412,1446,1450,1454,1458,1476,1518,1526,1527,1566,1569,1626,1641,1693,1702,1724,1729,1754,1769,1788,1800,1804,1813,1816,1823,1916,1918,1938,1959,1960,1965,1998,2029,2050,2120,2137,2152,2174,2286,2297,2312,2341,2347,2351,2352,2378,2421,2454,2460,2473,2507,2527,2533,2556,2571,2655,2660,2669,2683,2760,2858,2860,2898,2899,2934,2939,2989,3018,3049,3050,3051]
index_remove = list(dict.fromkeys(index_remove))
dataset = pd.read_pickle(os.path.join(os.getcwd(), "dataset_58,000_14012020_latest_version.pkl"))
dataset_twelve = dataset.iloc[55000:].reset_index(drop=True)
dataset_twelve = dataset_twelve[~dataset_twelve.index.isin(index_remove)]
dataset_twelve.shape

In [ ]:
"""
Loading the data extracted.
Those are the most important data source across all the notebook. 
The whole data extraction part was executed to end up with the following 6 files per mini-batch.

The examiner can directly import the files below to review the movie content extracted for the twelfth mini-batch.
"""
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_twelve\\plot_twelve_20012020.pkl'), 'rb') as f:
    
    plot = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_twelve\\rating_twelve_20012020.pkl'), 'rb') as f:
    
    rating = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_twelve\\actors_twelve_20012020.pkl'), 'rb') as f:
    
    actors = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_twelve\\director_twelve_20012020.pkl'), 'rb') as f:
    
    director = pickle.load(f)
        
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_twelve\\reviews_twelve_24012020.pkl'), 'rb') as f:
    
    reviews = pickle.load(f)
    
assert len(plot) == len(rating) == len(actors) == len(director) == len(reviews)

In [ ]:
dataset_twelve['actors'] = actors
dataset_twelve['plot'] = plot
dataset_twelve['imdb_rating'] = rating
dataset_twelve['director'] = director
dataset_twelve['reviews'] = reviews
dataset_twelve = dataset_twelve.drop(["movieId", "imdbId", "synopsis_url"], axis=1)

In [ ]:
index_remove=[]
index_remove=[2,44,46,53,66,82,85,91,104,109,157,168,174,175,176,204,207,211,218,220,224,225,232,246,270,272,275,290,294,307,351,361,372,387,406,439,468,470,477,480,502,512,521,525,527,536,551,553,558,560,564,578,581,588,589,595,597,598,599,600,614,615,616,618,619,620,621,622,623,629,639,645,646,647,657,658,663,666,676,682,685,691,692,693,695,697,698,699,700,701,703,705,709,711,712,713,714,716,717,718,720,721,723,724,727,728,730,731,733,734,736,739,743,747,749,750,751,754,755,757,762,782,792,795,797,798,800,801,806,808,809,812,814,815,821,822,834,835,836,837,848,849,850,852,864,867,869,874,878,883,897,907,908,912,926,934,944,945,948,949,958,961,962,971,974,976,1006,1010,1017,1028,1054,1059,1069,1074,1079,1080,1081,1082,1086,1096,1097,1098,1111,1113,1115,1122,1125,1129,1152,1153,1184,1187,1194,1204,1216,1224,1242,1243,1255,1261,1267,1277,1281,1282,1284,1292,1302,1306,1313,1316,1322,1323,1332,1335,1350,1351,1354,1362,1367,1376,1378,1379,1381,1390,1392,1393,1439,1440,1441,1442,1443,1458,1463,1479,1481,1489,1507,1518,1527,1541,1546,1548,1550,1565,1566,1567,1568,1576,1585,1586,1594,1600,1603,1614,1615,1623,1635,1636,1637,1648,1650,1652,1657,1659,1662,1664,1697,1698,1699,1723,1725,1741,1742,1746,1757,1762,1768,1769,1770,1772,1793,1794,1795,1796,1798,1801,1805,1810,1811,1816,1842,1854,1871,1874,1882,1885,1917,1933,1934,1936,1938,1943,1944,1946,1960,1980,1981,1989,1990,1993,2014,2020,2045,2049,2052,2055,2063,2071,2078,2082,2092,2110,2112,2118,2120,2127,2128,2130,2131,2137,2141,2144,2149,2151,2162,2165,2177,2183,2189,2192,2193,2196,2197,2199,2201,2206,2213,2216,2217,2218,2224,2225,2241,2257,2260,2273,2281,2298,2315,2318,2321,2322,2332,2365,2382,2397,2416,2421,2435,2437,2438,2440,2442,2443,2445,2447,2455,2467,2469,2481,2482,2486,2496,2502,2509,2510,2511,2519,2522,2523,2525,2545,2555,2557,2558,2569,2571,2575,2585,2586,2587,2598,2602,2604,2606,2610,2617,2618,2621,2626,2627,2634,2635,2638,2639,2640,2647,2648,2657,2659,2661,2664,2665,2673,2677,2679,2684,2687,2691,2692,2693,2694,2697,2721,2722,2725,2726,2731,2738,2740,2747,2751,2754,2757,2763,2774,2785,2795,2804,2816,2818,2820]
dataset_twelve = dataset_twelve[~dataset_twelve.index.isin(index_remove)]
dataset_twelve = dataset_twelve[dataset_twelve.astype(str)['reviews'] != '[]']
dataset_twelve.shape

In [ ]:
"""
This is the table to use in Part 1, and the table that will be concatenated with the rest of the mini-batch tables.
"""
# dataset_twelve.to_pickle(os.path.join(os.getcwd(), "58,000 movies\\movie_twelve\\dataset_twelve_final_25012020.pkl"))